### Notebook for concept detection in neural network

In [15]:
%load_ext autoreload
%autoreload 2

In [12]:
import numpy as np
import os
import sys

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

from concepts import static_concepts, linear_regression, generate_static_concept_datasets
from policy import ConvNet, ResNet
from utils import concept_folder_setup_and_score

import tensorflow as tf

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

reward_function = "zero_sum" # "concept_fn", "zero_sum" or "jem"
model_name = "net"
session_name = "probing"
board_size = 7
board_name = f'{board_size}x{board_size}'
resnet = True

model_type = "resnet" if resnet else "convnet"

agents_to_sample = [0, 10, 50, 100, 300, 500, 600, 800, 1000]

full_model_path = f"../models/saved_sessions/{model_type}/{reward_function}/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = static_concepts.number_of_legal_moves

CONCEPT_NAME = static_concepts.number_of_legal_moves.__name__

BINARY = False

CASES_TO_SAMPLE = 2500 # 25000

SAMPLE_RATIO = 0.5

# Load the models
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, model_path)
    else:
        model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [13]:
positive_cases, negative_cases = generate_static_concept_datasets(CASES_TO_SAMPLE, agents, board_size, CONCEPT_FUNC, sample_ratio=SAMPLE_RATIO, binary=BINARY)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases) # Target cases if continuous

Continues cases for concept 'number_of_legal_moves': 3590it [05:14, 11.41it/s]


In [14]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 5, 7, 7)
Negative cases:  (2500,)


In [4]:
if not BINARY:
    print("Continuous case - skipping duplicates check")
else:
    # Test if there are any duplicates accross the two sets
    for i in range(len(positive_cases)):
        for j in range(len(negative_cases)):
            if np.array_equal(positive_cases[i], negative_cases[j]):
                print("Duplicate found!")
                print(positive_cases[i])
                print(negative_cases[j])
                break

In [3]:
if not BINARY:
    print("Continuous case - skipping duplicates check")
else:
    # Find the number of any duplicates within the positive cases
    duplicate_count = 0
    for i in range(len(positive_cases)):
        for j in range(i + 1, len(positive_cases)):
            if np.array_equal(positive_cases[i], positive_cases[j]):
                duplicate_count += 1
                break

    print("Duplicate count in positive cases: ", duplicate_count)

    # Find the number of any duplicates within the negative cases
    duplicate_count = 0
    for i in range(len(negative_cases)):
        for j in range(i + 1, len(negative_cases)):
            if np.array_equal(negative_cases[i], negative_cases[j]):
                duplicate_count += 1
                break

    print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  1
Duplicate count in negative:  13


In [ ]:
from env import gogame
# Print all the positive cases
for i in range(100, 120):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [11]:
from env import gogame

if not BINARY:
    # Print random cases from the negative cases
    for i in range(100, 120):
        print(positive_cases[i])
        print(f'Target: {negative_cases[i]}')
        print(gogame.str(positive_cases[i], nn_format=True))
        print()

[[[1. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 1. 1.]
  [0. 0. 1. 1. 0. 1. 1.]
  [1. 0. 1. 1. 1. 1. 1.]
  [0. 1. 1. 1. 1. 0. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]

 [[1. 0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 1. 1.]
  [0. 0. 0. 1. 0. 1. 1.]
  [1. 0. 1. 1. 1. 1. 1.]
  [0. 1. 1. 1. 1. 0. 1.]
  [1. 1. 1. 1. 1. 1. 1.]]

 [[0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]]]
Target: 29.0
	0 1 2 3 4 5 6 
0	○═╤═╤═╤═╤═●═○
1	○─┼─┼─┼─●─●─╢
2	╟─┼─┼─○─┼─○─○
3	╟─┼─○─○─┼─○─○
4	○─●─○─○─○─○─○
5	╟─○─

In [15]:
# Positions to consider are 80% of the total positions
if BINARY:
    POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
    print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
    #POSITIONS_TO_CONSIDER = 4000 #40000
    VALIDATION_POSITIONS = 10000 #10000
else:
    POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0])
    print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
    #POSITIONS_TO_CONSIDER = 4000 #40000
    VALIDATION_POSITIONS = 10000

epochs = 5

Positions to consider: 2000


In [18]:
# First test if the concept can be regressed form the inputs
name = "input"
if BINARY:
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
else:
    all_cases = np.array(positive_cases)
    all_labels = negative_cases

all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY,
    epochs=epochs,
    verbose=1
)

score = 0 if score < 0 else score

print("Regression score: ", score)

concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, name, score)

Regression score:  0.9959945199475025


In [19]:
from tqdm import tqdm
epochs_to_look_at = agents_to_sample

bar = tqdm(total=len(epochs_to_look_at), desc="Epochs")

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    if resnet:
        model = ResNet(board_size, path)
    else:
        model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    if BINARY:
        all_cases = np.concatenate([positive_cases, negative_cases])
        all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    else:
        all_cases = np.array(positive_cases)
        all_labels = negative_cases

    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=epochs,
            verbose=1
        )
        score = 0 if score < 0 else score
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
        
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    concept_folder_setup_and_score('static', model_type, board_name, session_name, CONCEPT_NAME, epoch, concept_presence_per_layer)

    bar.update()

Getting activation outputs: 100%|██████████| 79/79 [00:02<00:00, 27.65it/s]


Performing regression for layer 0
The presence of number_of_legal_moves in resblock 0 is 0.9952108197962243
Performing regression for layer 1
The presence of number_of_legal_moves in resblock 1 is 0.9946240029018311
Performing regression for layer 2
The presence of number_of_legal_moves in resblock 2 is 0.9953593018276341
Performing regression for layer 3
The presence of number_of_legal_moves in resblock 3 is 0.9930366251099038
Performing regression for layer 4
The presence of number_of_legal_moves in resblock 4 is 0.9955431789411405
Performing regression for layer 5


The presence of number_of_legal_moves in resblock 5 is 0.993293279590505
Performing regression for layer 6
The presence of number_of_legal_moves in resblock 6 is 0.6821716908266136
Performing regression for layer 7
The presence of number_of_legal_moves in resblock 7 is 0.8932924319324594


Getting activation outputs: 100%|██████████| 79/79 [00:02<00:00, 30.72it/s]


Performing regression for layer 0
The presence of number_of_legal_moves in resblock 0 is 0.9954864035568581
Performing regression for layer 1
The presence of number_of_legal_moves in resblock 1 is 0.9942071099779863
Performing regression for layer 2
The presence of number_of_legal_moves in resblock 2 is 0.9955627506256405
Performing regression for layer 3
The presence of number_of_legal_moves in resblock 3 is 0.9936125323617144
Performing regression for layer 4
The presence of number_of_legal_moves in resblock 4 is 0.9951176126275509
Performing regression for layer 5


...

Performing regression for layer 0
The presence of number_of_legal_moves in resblock 0 is 0.991990187999679
Performing regression for layer 1
The presence of number_of_legal_moves in resblock 1 is 0.9876430197338155
Performing regression for layer 2
The presence of number_of_legal_moves in resblock 2 is 0.9857409135053519
Performing regression for layer 3
The presence of number_of_legal_moves in resblock 3 is 0.9735902182921191
Performing regression for layer 4
The presence of number_of_legal_moves in resblock 4 is 0.979959564122784
Performing regression for layer 5


The presence of number_of_legal_moves in resblock 5 is 0.9517409583442961
Performing regression for layer 6
The presence of number_of_legal_moves in resblock 6 is 0.597894143669737
Performing regression for layer 7
The presence of number_of_legal_moves in resblock 7 is 0.894153289002078


Getting activation outputs: 100%|██████████| 79/79 [00:03<00:00, 24.41it/s]


Performing regression for layer 0
The presence of number_of_legal_moves in resblock 0 is 0.9918431904205289
Performing regression for layer 1
The presence of number_of_legal_moves in resblock 1 is 0.9887262795051849
Performing regression for layer 2
The presence of number_of_legal_moves in resblock 2 is 0.9868713100353691
Performing regression for layer 3
The presence of number_of_legal_moves in resblock 3 is 0.9708156175362783
Performing regression for layer 4
The presence of number_of_legal_moves in resblock 4 is 0.9800539750471354
Performing regression for layer 5


The presence of number_of_legal_moves in resblock 5 is 0.9587307536442622
Performing regression for layer 6
The presence of number_of_legal_moves in resblock 6 is 0.6518560660972745
Performing regression for layer 7
The presence of number_of_legal_moves in resblock 7 is 0.8930294959368852
